In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../metnet')

In [3]:
from metnet import MetNet
import torch
import torch.nn.functional as F


In [4]:
model = MetNet(
        hidden_dim=32,
        forecast_steps=10, # should be 24 timesteps out
        input_channels=1, # 12 timeteps in
        output_channels=1, # 1 data channel in
        sat_channels=1, # 1 data channel in
        input_size=32, # =128/4, where 128 is the image dimensions
)

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [6]:
model = model.to(DEVICE)

In [7]:
# input_size = (1, 24, 1, 64, 64)
# torchinfo.summary(model, input_size)

In [7]:
sum(p.numel() for p in model.parameters())

1823649

In [8]:
# MetNet expects original HxW to be 4x the input size
# x = torch.randn((2, 12, 16, 128, 128))
x = torch.randn((1, 12, 1, 128, 128)).to(DEVICE)
out = model(x)

/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [9]:
out.shape

torch.Size([1, 10, 1, 64, 64])

In [10]:
# Modified MetNet creates predictions for the center 1/2
y = torch.randn((1, 10, 1, 64, 64)).to(DEVICE)

In [11]:
F.mse_loss(out, y).backward()